In [1]:
import matplotlib.pylab as plt
import numpy as np

from landlab import HexModelGrid, imshow_grid
from landlab.components import FlowAccumulator, LinearDiffuser, FastscapeEroder, ChannelProfiler

In [2]:
#grid = HexModelGrid((20, 10), 50)

nr = 50
nc = 30
dx = 50

np.random.seed(42)
grid1 = HexModelGrid((nr, nc), dx)
grid2 = HexModelGrid((nr, nc), dx)
grid3 = HexModelGrid((nr, nc), dx)

initial_noise = np.random.rand(grid1.core_nodes.size)

z1 = grid1.add_zeros("topographic__elevation", at="node")
z1[grid1.core_nodes] += initial_noise

z2 = grid2.add_zeros("topographic__elevation", at="node")
z2[grid2.core_nodes] += initial_noise

z3 = grid3.add_zeros("topographic__elevation", at="node")
z3[grid3.core_nodes] += initial_noise

In [3]:
U = 0.001
D = 0.01

K_min = 0.00001
K_max = 0.001
max_slp = 0.3

In [4]:
K1 = grid1.add_ones("water_erodibility", at="node")
K1[:] = (K_min + K_max) / 2.

K2 = grid2.add_ones("water_erodibility", at="node")
K2[:] = 0.0001

K3 = grid3.add_ones("water_erodibility", at="node")
K3[:] = 0.0001

In [5]:
fa1 = FlowAccumulator(grid1)
sp1 = FastscapeEroder(grid1, K_sp="water_erodibility")
ld1 = LinearDiffuser(grid1, linear_diffusivity=D)

fa2 = FlowAccumulator(grid2)
sp2 = FastscapeEroder(grid2, K_sp="water_erodibility")
ld2 = LinearDiffuser(grid2, linear_diffusivity=D)

fa3 = FlowAccumulator(grid3)
sp3 = FastscapeEroder(grid3, K_sp="water_erodibility")
ld3 = LinearDiffuser(grid3, linear_diffusivity=D)

In [ ]:
total_time = 1e6
dt = 100
nts = int(total_time / dt)

for i in range(nts):

    # topographic__steepest_slope is update in fa.r1s()
    fa1.run_one_step()
    fa2.run_one_step()
    fa3.run_one_step()

    # now we want to update K.
    # say that a waterfall is anything above 15 degrees (or max_slp of 0.3)
    # When slope is 0 lets make K = K_min
    # When slope is max_slp or above lets make K = K_max
    # We could make K continue to increase, but it might get way too
    # fast... so we'll threshold it for the moment

    slp2 = grid2.at_node["topographic__steepest_slope"].copy()
    slp2[slp2 > max_slp] = max_slp

    grid2.at_node["water_erodibility"][:] = K_min + (K_max -
                                                     K_min) * slp2 / max_slp

    slp3 = grid3.at_node["topographic__steepest_slope"].copy()
    slp3[slp3 > max_slp] = max_slp

    grid3.at_node["water_erodibility"][:] = K_max + (K_min -
                                                     K_max) * slp3 / max_slp

    sp1.run_one_step(dt)
    ld1.run_one_step(dt)
    z1[grid1.core_nodes] += U * dt

    sp2.run_one_step(dt)
    ld2.run_one_step(dt)
    z2[grid2.core_nodes] += U * dt

    sp3.run_one_step(dt)
    ld3.run_one_step(dt)
    z3[grid3.core_nodes] += U * dt

In [ ]:
plt.plot(grid1.at_node["topographic__steepest_slope"][grid1.core_nodes],
         grid1.at_node["water_erodibility"][grid1.core_nodes],
         ".",
         label="baseline")
plt.plot(grid2.at_node["topographic__steepest_slope"][grid2.core_nodes],
         grid2.at_node["water_erodibility"][grid2.core_nodes],
         ".",
         label="with increase in K with slope")
plt.plot(grid3.at_node["topographic__steepest_slope"][grid2.core_nodes],
         grid3.at_node["water_erodibility"][grid2.core_nodes],
         ".",
         label="with decrease in K with slope")
plt.legend()

In [ ]:
elev_max = max(z1.max(), z2.max(), z3.max())

plt.figure()
imshow_grid(grid1,
            z1,
            plot_name="baseline",
            vmin=0,
            vmax=elev_max,
            cmap='gist_earth')
plt.figure()
imshow_grid(grid2,
            z2,
            plot_name="with increase in K with slope",
            vmin=0,
            vmax=elev_max,
            cmap='gist_earth')

plt.figure()
imshow_grid(grid3,
            z3,
            plot_name="with decrease in K with slope",
            vmin=0,
            vmax=elev_max,
            cmap='gist_earth')

In [ ]:
slp_vmax = max(grid1.at_node["topographic__steepest_slope"].max(),
               grid2.at_node["topographic__steepest_slope"].max(),
               grid3.at_node["topographic__steepest_slope"].max())

plt.figure()
imshow_grid(grid1,
            "topographic__steepest_slope",
            plot_name="baseline",
            vmin=0,
            vmax=slp_vmax)
plt.figure()
imshow_grid(grid2,
            "topographic__steepest_slope",
            plot_name="with increase in K with slope",
            vmin=0,
            vmax=slp_vmax)
plt.figure()
imshow_grid(grid3,
            "topographic__steepest_slope",
            plot_name="with decrease in K with slope",
            vmin=0,
            vmax=slp_vmax)

In [ ]:
cp1 = ChannelProfiler(grid1, number_of_watersheds=4, main_channel_only=True)
cp1.run_one_step()

cp2 = ChannelProfiler(grid2, number_of_watersheds=4, main_channel_only=True)
cp2.run_one_step()

cp3 = ChannelProfiler(grid3, number_of_watersheds=4, main_channel_only=True)
cp3.run_one_step()

fig, ax = plt.subplots(nrows=2, ncols=3)

plt.axes(ax[0, 0])
cp1.plot_profiles()
ax[0, 0].set_title("baseline")
ax[0, 0].set_ylim([0, elev_max])

plt.axes(ax[0, 1])
cp2.plot_profiles()
ax[0, 1].set_title("with increase in K with slope")
ax[0, 1].set_ylim([0, elev_max])

plt.axes(ax[0, 2])
cp2.plot_profiles()
ax[0, 1].set_title("with decrease in K with slope")
ax[0, 1].set_ylim([0, elev_max])

plt.axes(ax[1, 0])
cp1.plot_profiles_in_map_view()

plt.axes(ax[1, 1])
cp2.plot_profiles_in_map_view()

plt.axes(ax[1, 2])
cp3.plot_profiles_in_map_view()

In [ ]:
plt.loglog(grid1.at_node["topographic__steepest_slope"][grid1.core_nodes],
           grid1.at_node["drainage_area"][grid1.core_nodes],
           ".",
           label="baseline")
plt.loglog(grid2.at_node["topographic__steepest_slope"][grid2.core_nodes],
           grid2.at_node["drainage_area"][grid2.core_nodes],
           ".",
           label="with increase in K with slope")
plt.loglog(grid3.at_node["topographic__steepest_slope"][grid2.core_nodes],
           grid3.at_node["drainage_area"][grid2.core_nodes],
           ".",
           label="with decrease in K with slope")
plt.legend()